# Đọc dữ liệu từ file PKL #

In [1]:
#Read *.pkl
import pickle

def read_file_pkl(pkl_file_path):
    with open(pkl_file_path,'rb') as f:
        data = pickle.load(f)
    return data

In [2]:
#Insert to database
from pymongo import MongoClient
import os
from tqdm import tqdm
from evaluate import load
import string
import unicodedata
import re

#Connect2Database
client = MongoClient("192.168.1.60", 27017)
collection = client['ASR']['ASR_All'] 
wer = load("wer")

def preprocess_text(text: str):
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    text = text.lower()
    text = unicodedata.normalize("NFKC", text)
    text = text.replace('\xa0', '')
    text = re.sub(r"\s{2,}", ' ',text);
    return text

def insert2database(record):
    time = len(record['array']) / 16000
    del record['array']
    record['visitted'] = False
    record['gold_sentence'] = ""
    record['labeled'] = False
    record['humman'] = None
    record['time'] = time
    record['location'] = None
    record['sex'] = None
    record['date_labeld'] = None
    record['wer_autosub_whisper'] = wer.compute(references=[preprocess_text(record['sentence'])],predictions=[preprocess_text(record['api_sentence'])])
    # if existing_record is None:
    if record['compare'] == True:   
        record['visitted'] = True
        record['gold_sentence'] = record['sentence']
        record['labeled'] = True
        record['humman'] = 'openAI/whisper'
    collection.insert_one(record)

count_equals = 0
count_all = 0
Equals_lst = []

for root, _, files in os.walk("Datasets/PublicDataset"):
    print(len(files))
    for file in files:
        print(file)
        path_file = os.path.join(root,file)
        data = read_file_pkl(path_file)
        for record in tqdm(data):
            if record['compare'] == True:
                Equals_lst.append(record)
            insert2database(record)
        count_all += len(data)
        
print('equals sample: ',str(len(Equals_lst)))
print('all sample: ',str(count_all))
print(len(Equals_lst)/count_all)   
client.close()

/home/nlp_team/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3
VLSP.pkl


100%|██████████| 23799/23799 [01:05<00:00, 361.20it/s]


VIVOS.pkl


100%|██████████| 12420/12420 [00:34<00:00, 360.51it/s]


FPT.pkl


100%|██████████| 25919/25919 [01:06<00:00, 388.39it/s]

equals sample:  13271
all sample:  62138
0.2135730149023142


# Create Dataset #

In [ ]:
from datasets import load_dataset

dataset = load_dataset("AIA_ASR", data_dir=".")

# Thêm Trường thời gian #

In [ ]:
from pymongo import MongoClient
import librosa
from tqdm import tqdm
client = MongoClient("192.168.1.60", 27017)
collection = client['ASR']['ASR']


random_samples = list(collection.find({},{"path": 1,"_id": 0}))
    #Update sample is visitted
count = 0
for sample in tqdm(random_samples):
    try:
        time = librosa.get_duration(filename=sample['path'])
        if time >= 5:
            count += 1
    except:
        print(sample['path'])
    # myquery = { "path": sample['path']}
    # update_data = { "$set": { "time": True } }
    # collection.update_one(myquery, update_data)
client.close()
print(count/len(random_samples))

# Lọc tập Test #

In [1]:
import json
import pandas as pd


with open('./Datalabeled_18_12.json','r',encoding='utf-8') as f:
    data_labeld = json.load(f)
    
df = pd.read_csv("./ASR_done_07_12.csv")
data_train_test = df['path']

In [ ]:
data_train_test = data_train_test.to_list()
data_labeld

In [4]:
len(data_labeld)

8542

In [9]:
for record in data_labeld:
    if record['path'] in data_train_test:
        data_labeld.remove(record)

In [11]:
with open('./Datalabeled_18_12.json','w',encoding='utf-8') as f:
    json.dump(data_labeld,f,ensure_ascii=False)

# Thêm trường WER-Score #

In [10]:
from pymongo import MongoClient
from tqdm import tqdm
from evaluate import load

client = MongoClient("192.168.1.60", 27017)
collection = client['ASR']['ASR_All'] 

wer = load("wer")

def update_wer():
    try:
        all_records = list(collection.find({}))
        for record in tqdm(all_records):
            score_wer = wer.compute(predictions=[record['sentence']],references=[record['api_sentence']])
            myquery = { "_id": record['_id']}
            update_data = { "$set": {
                                "wer_autosub_whisper": score_wer
                                }}
            collection.update_one(myquery,update_data)
    except:
        client.close()
        
update_wer()
client.close()



100%|██████████| 1242741/1242741 [44:12<00:00, 468.52it/s] 


In [2]:
#Read *.pkl
import pickle

def read_file_pkl(pkl_file_path):
    with open(pkl_file_path,'rb') as f:
        data = pickle.load(f)
    return data

In [ ]:
data = read_file_pkl('/mnt/wsl/PHYSICALDRIVE0p1/toan/data_speech_to_text/ASR/pkls_done/VLSP.pkl')
data[0]

In [1]:
from pymongo import MongoClient
import os
from tqdm import tqdm
from evaluate import load
import string
import unicodedata
import re

#Connect2Database
client = MongoClient("192.168.1.60", 27017)
collection = client['ASR']['ASR_All'] 

/home/nlp_team/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
sample_stage = {"$sample": {"size": 100}}
random_samples = list(collection.aggregate([sample_stage]))

In [29]:
str(random_samples[0]['_id'])

'6585087aef952e214a10c222'

In [ ]:
from bson import ObjectId
target_object_id = ObjectId("6585087aef952e214a10c222")
result = collection.find_one({"_id": target_object_id})
result

# Thay đổi visited với audi dài #

In [1]:
from pymongo import MongoClient
from tqdm import tqdm
from evaluate import load
import random

client = MongoClient("192.168.1.60", 27017)
collection = client['ASR']['ASR'] 

wer = load("wer")

def update_visted():
    try:
        all_records = list(collection.find({"compare": False, "visitted": False, "time": {"$gte":5 }, "path": {"$regex": "Đời_Người_Tị_Nạn"}}))
        for record in tqdm(all_records):
            if random.random() > 0.1:
                myquery = { "_id": record['_id']}
                update_data = { "$set": {
                                    "visitted": True
                                    }}
                collection.update_one(myquery,update_data)
    except:
        client.close()
        
update_visted()
client.close()

/home/nlp_team/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 3145/3145 [00:00<00:00, 4244.81it/s]
